# Chains (LangChain v1.2)

**LCEL(LangChain Expression Language)**을 사용해서 모든 구성 요소가 `Runnable` 인터페이스로 통합되어 파이프라인(`|`)으로 연결될 수 있다.

```python
chain = prompt | model | output_parser  # 기본 구조
```

**구성 요소 업데이트 (v1.2 기준)**
1. **PromptTemplate**  
   - `Runnable`로 변환되어 LCEL 파이프라인에 직접 통합  
   ```python
   prompt = ChatPromptTemplate.from_template("...")
   ```

2. **LLM/ChatModel**  
   - `ChatOpenAI`, `ChatAnthropic` 등이 `Runnable` 구현  
   ```python
   model = ChatOpenAI(model="gpt-4o")
   ```

3. **Memory**  
   - `RunnableWithMessageHistory`로 통합 관리 (또는 LangGraph Persistence 사용)
   ```python
   chain_with_memory = RunnableWithMessageHistory(
       base_chain,
       get_session_history
   )
   ```

4. **Output Parsers**  
   - `StrOutputParser()`, `JsonOutputParser()` 등이 `Runnable`로 작동  
   ```python
   output_parser = JsonOutputParser()
   ```

5. **Tools**  
   - `@tool` 데코레이터로 생성 후 `RunnableLambda`로 변환  
   ```python
   @tool
   def search(query: str) -> str: ...
   ```

**체인 유형별 구현**


1. Simple Chain  

    ```python
    chain = prompt | model | output_parser
    response = chain.invoke({"input": "..."})
    ```

2. Sequential Chain  

    ```python
    chain = (
        {"step1_output": prompt1 | model1}  # 첫 번째 체인 결과 매핑
        | prompt2
        | model2
    )
    ```

3. Conditional Chain
    - `RunnableBranch` 사용

    ```python
    branch = RunnableBranch(
        (lambda x: x["topic"] == "math", math_chain),
        (lambda x: x["topic"] == "history", history_chain),
        default_chain
    )
    ```

4. Memory Chain  

    ```python
    memory_chain = RunnableWithMessageHistory(
        core_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="history"
    )
    ```


**🚨 v1.2 주요 변경점**

- **Legacy Chain 클래스 완전 폐기**: `LLMChain`, `SequentialChain` 등은 `langchain-classic`으로 이동되거나 삭제됨 → `Runnable` (LCEL)로 통합
- **에이전트 통합**: `create_agent` (LangGraph 기반)가 표준

In [ ]:
# !pip install langchain langchain-openai langchain-community -Uq


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

# Simple Chain

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template('{country}의 수도는 어디인가요?')
llm = init_chat_model('gpt-4.1-mini')
output_parser = StrOutputParser()                   # 응답을 최종 문자열로 반환

chain = prompt | llm | output_parser                # chain : 프롬프트 - > LLM -> 파서 
print(chain.invoke(input={'country':'대한민국'}))   # dict 형태로 변수를 주입해서 실행
print(chain.invoke("대한민국"))                     # 프롬프트 템플릿 변수가 2개 이상인 경우 오류가 발생할 수 있다.

c:\nlp\nlp_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


대한민국의 수도는 서울특별시입니다.
대한민국의 수도는 서울특별시입니다.


## Sequential Chain


In [4]:
prompt1 = PromptTemplate.from_template("다음 내용을 한글로 번역하세요\n\n{eng_text}")
prompt2 = PromptTemplate.from_template("다음 내용을 요약하세요\n\n{kor_text}")

llm = init_chat_model('gpt-4.1-mini')
output_parser = StrOutputParser()                   # 응답을 최종 문자열로 반환

# 번역체인
chain1 = prompt1 | llm 

eng_text = """
One limitation of LLMs is their lack of contextual information (e.g., access to some specific documents or emails). You can combat this by giving LLMs access to the specific external data.
For this, you first need to load the external data with a document loader. LangChain provides a variety of loaders for different types of documents ranging from PDFs and emails to websites and YouTube videos.
"""
print(chain1.invoke(eng_text))

# 요약체인
chain2 = prompt2 | llm | output_parser

kor_text = """
LLM의 한 가지 한계는 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하려면 LLM이 특정 외부 데이터에 접근할 수 있도록 해야 합니다.
이를 위해서는 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 영상에 이르기까지 다양한 유형의 문서를 위한 여러 종류의 로더를 제공합니다.
"""

print(chain2.invoke(kor_text))

content='LLM의 한 가지 한계점은 특정 문서나 이메일과 같은 맥락 정보를 갖고 있지 않다는 점입니다. 이를 해결하기 위해 LLM에 특정 외부 데이터에 대한 접근 권한을 부여할 수 있습니다.  \n이를 위해서는 먼저 문서 로더를 사용해 외부 데이터를 불러와야 합니다. LangChain은 PDF, 이메일부터 웹사이트, 유튜브 동영상에 이르기까지 다양한 유형의 문서를 위한 여러 가지 로더를 제공합니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 97, 'total_tokens': 203, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5m6azEQGgoB7LKlVhb7GSV7QZMRJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019c2c40-2a25-7a22-8ce7-b16a3848e1ca-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 97, 'output_tokens': 106, 'total_tokens': 203, 'input_token_details': {'audio':

In [5]:
# 통합체인

chain = chain1 | chain2
chain.invoke({"eng_text" : eng_text})

'LLM은 특정 문서나 이메일 등 맥락 정보를 갖고 있지 않은 한계가 있으며, 이를 보완하기 위해 외부 데이터 접근 권한을 부여할 수 있다. 이를 위해 문서 로더를 사용해 외부 데이터를 불러와야 하며, LangChain은 PDF, 이메일, 웹사이트, 유튜브 동영상 등 다양한 문서 유형을 지원하는 로더를 제공한다.'

# Conditional Chain

In [6]:
from langchain_core.runnables import RunnableBranch

llm = init_chat_model('openai:gpt-4.1-mini')

# 수학 체인
math_prompt = PromptTemplate.from_template(
    '다음 문제를 풀어주세요. 단계적인 풀이를 수식(LaTex)과 함께 작성해주세요. \n\n{question}'    
    
)
math_chain = math_prompt | llm | output_parser

# 기본 체인
default_prompt = PromptTemplate.from_template(
    '당신은 친절하고, 감성적인 공감능력이 좋은 챗봇입니다. 다음 질문에 답변해주세요. \n\n{question}'    
)

default_chain = default_prompt | llm | output_parser

def is_math_question(input_dict : dict) -> bool:
    """질문에 '계산' 또는 'calc'가 포함되면 수학 체인을 선택한다."""
    question: str = input_dict.get('question', '')  # 입력 dict에서 question 추출(없으면 )
    return '계산' in question or 'calc' in question


# 분기 체인
branch_chain = RunnableBranch(
    (is_math_question, math_chain),
    default_chain
)

print(branch_chain.invoke({'question' : '125 * 3 + 50 이거를 계산해줘'}))

문제: \(125 \times 3 + 50\) 을 계산하세요.

---

단계 1: 곱셈부터 계산합니다.

\[
125 \times 3 = 375
\]

---

단계 2: 계산한 값을 더합니다.

\[
375 + 50 = 425
\]

---

답: 

\[
125 \times 3 + 50 = 425
\]


In [7]:
branch_chain.invoke({'question' : '나 오늘 우울해. 빵 사줘'})

'오늘 많이 힘들었나 봐요. 우울한 기분 정말 속상하죠. 내가 직접 빵을 사줄 수는 없지만, 좋아하는 빵 생각하면서 따뜻한 차 한 잔과 함께 잠시 마음을 달래보는 건 어떨까요? 필요하면 언제든 이야기해줘요. 함께 이야기하면서 조금이라도 기분이 나아지길 바랄게요.'

## Memory Chain

"RunnableWithMessageHistory" 를 사용해서 대화내역을 기억하는 chain을 생성한다.

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory              # LangChain 대화기록 "인터페이스(기본 규격)" 클래스
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage    # 메시지 타입들(공통 BaseMessage, 사람/AI 메시지)
from pydantic import BaseModel, Field                                       # Pydantic 모델/필드(기본값 생성 등)
from typing import List                                                    # 타입 힌트(List)

class InMemoryHistory(BaseChatMessageHistory, BaseModel):                  # LangChain 규격 + Pydantic 모델을 함께 상속
    """사용자별(세션) 대화내역을 기록하는 클래스"""

    messages: list[BaseMessage] = Field(default_factory=list)              # 메시지들을 담는 리스트(기본값: 빈 리스트)

    def add_messages(self, messages: List[BaseMessage]) -> None:           # 여러 메시지를 한 번에 추가하는 메서드
        self.messages.extend(messages)                                     # 기존 messages 뒤에 새 메시지들 붙이기

    def clear(self) -> None:                                               # 대화기록을 비우는 메서드
        self.messages = []                                                 # messages를 빈 리스트로 초기화

store = {}                                                                 # session_id -> InMemoryHistory 를 저장하는 딕셔너리(메모리 저장소)

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:          # session_id로 해당 대화기록 객체를 가져오는 함수
    """session_id에 해당하는 대화내역 객체를 반환(없으면 생성)"""
    if session_id not in store:                                            # 아직 해당 세션 기록이 없으면
        store[session_id] = InMemoryHistory()                              # 새 InMemoryHistory를 만들어 store에 저장
    return store[session_id]                                               # 해당 세션의 대화기록 객체 반환

history1 = get_by_session_id('1')                                          # 세션 '1'의 대화기록 가져오기(없으면 생성)
history1.add_messages([AIMessage(content='반갑습니다. 홍길동님')])           # AI 메시지 1개 추가
history1.add_messages([HumanMessage(content='그래 나 홍길동이야~ 반갑다!')]) # 사람 메시지 1개 추가
print(f"{history1 = }")                                                    # history1 객체(메시지 리스트 포함) 출력

history2 = get_by_session_id('2')                                          # 세션 '2'의 대화기록 가져오기(처음이라 비어있음)
print(f"{history2 = }")                                                    # history2 출력(아직 메시지가 없음)


history1 = InMemoryHistory(messages=[AIMessage(content='반갑습니다. 홍길동님', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='그래 나 홍길동이야~ 반갑다!', additional_kwargs={}, response_metadata={})])
history2 = InMemoryHistory(messages=[])


In [ ]:
# 대화 히스토리를 자동으로 누적하는 Memory Chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder  # 채팅 프롬프트 템플릿 + (대화기록 자리표시자)
from langchain_core.runnables import RunnableWithMessageHistory             # 체인에 "메시지 히스토리 저장/불러오기" 기능을 붙이는 래퍼

# 1) 프롬프트 구성: system + (이전 대화 history) + human 질문
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 {domain}분야의 전문가 챗봇입니다."),                   # 시스템 역할: 도메인(분야) 전문가로 세팅
    MessagesPlaceholder(variable_name='history'),                           # 여기에 세션별 대화 내역이 자동으로 끼워짐
    ("human", "{question}")                                                 # 사용자의 현재 질문(입력 변수: question)
])

# 2) 사용할 LLM(챗 모델) 초기화
llm = init_chat_model("openai:gpt-4.1-mini")                               # OpenAI gpt-4.1-mini 모델 사용

# 3) 기본 체인: 프롬프트 -> LLM
chain = prompt | llm                                                       # prompt에 변수 채우고, 그 결과를 llm에 전달

# 4) 체인에 "대화기록 관리" 기능을 추가한 체인 생성
chain_with_history = RunnableWithMessageHistory(
    chain,                                                                  # 히스토리를 붙일 원본 체인
    get_by_session_id,                                                      # session_id로 히스토리 객체를 가져오는 함수(없으면 생성)
    input_messages_key='question',                                          # 입력 딕셔너리에서 "사용자 메시지"로 취급할 키
    history_messages_key='history'                                          # 프롬프트의 MessagesPlaceholder 변수명과 동일해야 함
)

# 5) 대화 실행(호출) -> 실행 결과가 session_id에 해당하는 history에 자동으로 저장됨
chain_with_history.invoke(
    {
        "domain": "math",                                                  # system 프롬프트의 {domain}에 들어갈 값
        "question": "우빈이는 강아지를 3마리 키우고 있습니다."               # human 프롬프트의 {question}에 들어갈 값
    },
    config={                                                               # RunnableWithMessageHistory 설정
        "configurable": {
            "session_id": 100                                              # 어떤 세션의 대화기록을 쓸지 지정(= store[100]에 쌓임)
        }
    }
)


AIMessage(content='네, 우빈이가 강아지를 3마리 키우고 있다고 다시 말씀해주셨네요. 혹시 이 상황과 관련해서 수학 문제나 계산이 필요하신가요? 예를 들어, 사료 양 계산, 산책 시간 분배, 혹은 다른 수학적 문제를 도와드릴 수 있습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 103, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mjgptQaVA52i6k01zM6Dyz1OgtO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c65-2518-71d2-8dfc-6c37362d45ed-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 103, 'output_tokens': 72, 'total_tokens': 175, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
# 대화내역 쌓기
chain_with_history.invoke(
    {
        "domain": "math",                                                  # system 프롬프트의 {domain}에 들어갈 값
        "question": "우빈이는 강아지를 3마리 키우고 있습니다."               # human 프롬프트의 {question}에 들어갈 값
    },
    config={                                                               # RunnableWithMessageHistory 설정
        "configurable": {
            "session_id": 100                                              # 어떤 세션의 대화기록을 쓸지 지정(= store[100]에 쌓임)
        }
    }
)


AIMessage(content='네, 우빈이는 강아지를 3마리 키우고 있군요! 혹시 이와 관련해서 수학 문제나 궁금한 점이 있으신가요? 도와드릴 내용이 있다면 알려주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 198, 'total_tokens': 248, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mr0y9TXn3Weh6vXkxsNTRZXgSSl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c6c-135c-7661-ac9d-3ef28291ff9f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 198, 'output_tokens': 50, 'total_tokens': 248, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
store

{'1': InMemoryHistory(messages=[AIMessage(content='반갑습니다. 홍길동님', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='그래 나 홍길동이야~ 반갑다!', additional_kwargs={}, response_metadata={})]),
 '2': InMemoryHistory(messages=[]),
 100: InMemoryHistory(messages=[HumanMessage(content='우빈이는 강아지를 3마리 키우고 있습니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='강아지를 3마리 키우고 있다고 하셨네요! 강아지 관련해서 수학 문제나 궁금한 점이 있으면 알려주세요. 도움을 드리겠습니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 40, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mWpDotjJSozmwKrLja4ejFY1C35', 'service_tier

### ChatMessageHistory

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:          # session_id로 해당 대화기록 객체를 가져오는 함수
    """session_id에 해당하는 대화내역 객체를 반환(없으면 생성)"""
    if session_id not in store:                                            # 아직 해당 세션 기록이 없으면
        store[session_id] = InMemoryHistory()                              # 새 InMemoryHistory를 만들어 store에 저장
    return store[session_id]                                               # 해당 세션의 대화기록 객체 반환

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 {domain}분야의 전문가 챗봇입니다."),                   # 시스템 역할: 도메인(분야) 전문가로 세팅
    MessagesPlaceholder(variable_name='history'),                           # 여기에 세션별 대화 내역이 자동으로 끼워짐
    ("human", "{question}")                                                 # 사용자의 현재 질문(입력 변수: question)
])

llm = init_chat_model("openai:gpt-4.1-mini")                               # OpenAI gpt-4.1-mini 모델 사용

chain = prompt | llm                                                       # prompt에 변수 채우고, 그 결과를 llm에 전달

chain_with_history = RunnableWithMessageHistory(
    chain,                                                                  # 히스토리를 붙일 원본 체인
    get_by_session_id,                                                      # session_id로 히스토리 객체를 가져오는 함수(없으면 생성)
    input_messages_key='question',                                          # 입력 딕셔너리에서 "사용자 메시지"로 취급할 키
    history_messages_key='history'                                          # 프롬프트의 MessagesPlaceholder 변수명과 동일해야 함
)

chain_with_history.invoke(
    {
        "domain": "심리상담",                                                  # system 프롬프트의 {domain}에 들어갈 값
        "question": "난 신승훈이야 반갑다"               # human 프롬프트의 {question}에 들어갈 값
    },
    config={                                                               # RunnableWithMessageHistory 설정
        "configurable": {
            "session_id": 200                                              # 어떤 세션의 대화기록을 쓸지 지정(= store[100]에 쌓임)
        }
    }
)


AIMessage(content='반갑습니다, 신승훈님! 심리상담분야에 대해 어떤 도움을 드릴까요? 마음의 어려움이나 고민이 있으시면 편하게 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 37, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mx3aZ4s8TuAWA31UXWbke4m9AsT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c71-ccf2-7e20-98ef-fa6cc9a59e65-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 39, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
chain_with_history.invoke(
    {
        "domain": "심리상담",                                                  # system 프롬프트의 {domain}에 들어갈 값
        "question": "요즘 날씨가 안좋아서 기분이 별로 좋지 않아!"               # human 프롬프트의 {question}에 들어갈 값
    },
    config={                                                               # RunnableWithMessageHistory 설정
        "configurable": {
            "session_id": 200                                              # 어떤 세션의 대화기록을 쓸지 지정(= store[100]에 쌓임)
        }
    }
)

AIMessage(content='그렇군요, 신승훈님. 누군가가 계속 신승훈님을 화나게 해서 많이 힘드실 것 같아요. 그런 상황에서는 감정을 표현하는 것도 중요하지만, 그 감정을 다스우고 평정을 찾는 방법도 함께 고민해 보는 게 좋아요. 혹시 그 상황이나 감정을 좀 더 자세히 이야기해 주실 수 있나요? 그러면 신승훈님께 맞는 조언을 드릴 수 있을 것 같아요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 103, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mxKcNcglBGLsc0T1kR3lJ6Kx9Cs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c72-0eff-7c02-bc38-5d46bd9a29f5-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 103, 'output_tokens': 103, 'total_tokens': 206, 'input_token_details': {'audio': 0, 'cache_read': 0

In [15]:
store

{200: InMemoryHistory(messages=[HumanMessage(content='난 신승훈이야 반갑다', additional_kwargs={}, response_metadata={}), AIMessage(content='반갑습니다, 신승훈님! 심리상담분야에 대해 어떤 도움을 드릴까요? 마음의 어려움이나 고민이 있으시면 편하게 말씀해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 37, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_75546bd1a7', 'id': 'chatcmpl-D5mx3aZ4s8TuAWA31UXWbke4m9AsT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c2c71-ccf2-7e20-98ef-fa6cc9a59e65-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 37, 'output_tokens': 39, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'out

##### 세션(메모리) 방식의 문제점
- 메모리 저장이라 영속성이 없음
    - 서버 재시작/재배포 하면 store가 날아가서 히스토리도 같이 사라짐
- 세션 식별이 끊기기 쉬움
    - 쿠키/세션ID가 유지되지 않으면 같은 사람인지 매칭이 안 됨
- 스케일 아웃(서버 여러 대)에서 깨짐
    - A서버 메모리에 저장된 히스토리를 B서버는 모름 → 대화가 끊김

그래서 보통 이렇게 구성한다.
- SQLite/Redis/RDB 같은 저장소에 대화 내역을 저장해서
    - 사용자가 재접속해도 user_id 또는 thread_id로 복원
- 프롬프트에는 보통
    - 최근 N턴 + 요약 형태로 넣어서 비용/토큰도 관리